In [1]:
pip install duckdb --upgrade

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 0.9/11.0 MB 29.1 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/11.0 MB 33.3 MB/s eta 0:00:01
   -------------- ------------------------- 4.1/11.0 MB 32.8 MB/s eta 0:00:01
   --------------------- ------------------ 5.9/11.0 MB 34.5 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 34.9 MB/s eta 0:00:01
   ---------------------------------- ----- 9.4/11.0 MB 35.3 MB/s eta 0:00:01
   ---------------------------------------  10.9/11.0 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 34.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import wrds
import duckdb
import pandas as pd

In [10]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [18]:
db.describe_table(library="compsamp_snapshot", table="wrds_csq_pit")

Approximately 607568 rows in compsamp_snapshot.wrds_csq_pit.


,name,nullable,type,comment
0,pitdate1,True,DATE,Point-in-Time Effective Date
1,pitdate2,True,DATE,Point-in-Time Observation End Date
2,gvkey,True,VARCHAR(6),Global Company Key
3,indfmt,True,VARCHAR(12),Industry Format
4,consol,True,VARCHAR(2),Level of Consolidation - Company Interim Descr...
...,...,...,...,...
615,xopty,True,DOUBLE PRECISION,Implied Option Expense
616,xrdq,True,DOUBLE PRECISION,Research and Development Expense
617,xrdy,True,DOUBLE PRECISION,Research and Development Expense
618,xsgaq,True,DOUBLE PRECISION,"Selling, General and Administrative Expenses"


In [19]:
db.describe_table(library="comp_snapshot", table="wrds_csq_pit")['type']

Approximately 9981955 rows in comp_snapshot.wrds_csq_pit.


0                DATE
1                DATE
2          VARCHAR(6)
3         VARCHAR(12)
4          VARCHAR(2)
            ...      
667    NUMERIC(19, 4)
668    NUMERIC(19, 4)
669    NUMERIC(19, 4)
670    NUMERIC(19, 4)
671    NUMERIC(19, 4)
Name: type, Length: 672, dtype: object

In [21]:
unique_types = db.describe_table(library="compsamp_snapshot", table="wrds_csq_pit")['type'].astype(str).drop_duplicates()

print(unique_types)

Approximately 607568 rows in compsamp_snapshot.wrds_csq_pit.
0                 DATE
2           VARCHAR(6)
3          VARCHAR(12)
4           VARCHAR(2)
5           VARCHAR(1)
6             SMALLINT
8     DOUBLE PRECISION
10          VARCHAR(3)
11             INTEGER
Name: type, dtype: object


In [28]:
# Create DuckDB connection'''
'''
duck_conn = duckdb.connect('wrds_data.db')


# Get schema information
schema_info = db.describe_table(library="compsamp_snapshot", table="wrds_csq_pit")

# Drop existing table if it exists
duck_conn.execute("DROP TABLE IF EXISTS wrds_csq_pit")

# Create table with exact schema mapping
create_table_sql = "CREATE TABLE wrds_csq_pit ("
columns = []
for _, row in schema_info.iterrows():
    col_name = row['name']
    wrds_type = str(row['type']).upper()
    
    if wrds_type == 'DATE':
        duck_type = 'DATE'
    elif wrds_type.startswith('VARCHAR'):
        duck_type = wrds_type
    elif wrds_type == 'SMALLINT':
        duck_type = 'SMALLINT'
    elif wrds_type == 'INTEGER':
        duck_type = 'INTEGER'
    elif 'DOUBLE PRECISION' in wrds_type:
        duck_type = 'DOUBLE'
    else:
        duck_type = 'VARCHAR'
        
    columns.append(f"{col_name} {duck_type}")

create_table_sql += ", ".join(columns) + ")"
duck_conn.execute(create_table_sql)

# Process in chunks
chunksize = 100000
offset = 0

while True:
    query = f"""
        select * 
        from compsamp_snapshot.wrds_csq_pit 
        LIMIT {chunksize} 
        OFFSET {offset}
    """
    
    try:
        chunk = db.raw_sql(query)
        
        if len(chunk) == 0:
            break
        
        # Handle date columns properly
        date_cols = schema_info[schema_info['type'].astype(str).str.upper() == 'DATE']['name'].tolist()
        for col in date_cols:
            if col in chunk.columns:
                # Convert dates to strings, handling special cases
                def convert_date(x):
                    if pd.isna(x):
                        return None
                    elif str(x) == '9999-12-31':
                        return 'infinity'
                    try:
                        return pd.to_datetime(x).strftime('%Y-%m-%d')
                    except:
                        return None
                
                chunk[col] = chunk[col].apply(convert_date)
        
        # Insert the chunk into DuckDB
        duck_conn.register('temp_chunk', chunk)
        duck_conn.sql('INSERT INTO wrds_csq_pit SELECT * FROM temp_chunk')
        
        offset += chunksize
        print(f"Processed {offset} rows")
        
    except Exception as e:
        print(f"Error at offset {offset}: {str(e)}")
        raise
'''



Approximately 607568 rows in compsamp_snapshot.wrds_csq_pit.
Processed 100000 rows
Processed 200000 rows
Processed 300000 rows
Processed 400000 rows
Processed 500000 rows
Processed 600000 rows
Processed 700000 rows


In [23]:
# Create DuckDB connection
duck_conn = duckdb.connect('wrds_data.db')



# Get schema information 
schema_info = db.describe_table(library="comp_snapshot", table="wrds_csq_pit")

# Get total row count first
total_rows = db.raw_sql("SELECT COUNT(*) FROM comp_snapshot.wrds_csq_pit").iloc[0,0]
print(f"Total rows to process: {total_rows}")

# Drop existing table if it exists
duck_conn.execute("DROP TABLE IF EXISTS wrds_csq_pit")

# Create table with exact schema mapping
create_table_sql = "CREATE TABLE wrds_csq_pit ("
columns = []
for _, row in schema_info.iterrows():
   col_name = row['name']
   wrds_type = str(row['type']).upper()
   
   if wrds_type == 'DATE':
       duck_type = 'DATE'
   elif wrds_type.startswith('VARCHAR'):
       duck_type = wrds_type
   elif wrds_type == 'SMALLINT':
       duck_type = 'SMALLINT'
   elif wrds_type == 'INTEGER':
       duck_type = 'INTEGER'
   elif 'DOUBLE PRECISION' in wrds_type:
       duck_type = 'DOUBLE'
   else:
       duck_type = 'VARCHAR'
       
   columns.append(f"{col_name} {duck_type}")

create_table_sql += ", ".join(columns) + ")"
duck_conn.execute(create_table_sql)

# Process in chunks
chunksize = 500000
offset = 0

while True:
   query = f"""
       select * 
       from comp_snapshot.wrds_csq_pit 
       LIMIT {chunksize} 
       OFFSET {offset}
   """
   
   try:
       chunk = db.raw_sql(query)
       
       if len(chunk) == 0:
           break
       
       # Handle date columns properly
       date_cols = schema_info[schema_info['type'].astype(str).str.upper() == 'DATE']['name'].tolist()
       for col in date_cols:
           if col in chunk.columns:
               # Convert dates to strings, handling special cases
               def convert_date(x):
                   if pd.isna(x):
                       return None
                   elif str(x) == '9999-12-31':
                       return '2200-01-01'  # New upper bound
                   try:
                       return pd.to_datetime(x).strftime('%Y-%m-%d')
                   except:
                       return None
               
               chunk[col] = chunk[col].apply(convert_date)
       
       # Insert the chunk into DuckDB
       duck_conn.register('temp_chunk', chunk)
       duck_conn.sql('INSERT INTO wrds_csq_pit SELECT * FROM temp_chunk')
       
       offset += chunksize
       print(f"Processed {min(offset, total_rows)} rows out of {total_rows} ({(min(offset, total_rows)/total_rows*100):.2f}%)")
       
   except Exception as e:
       print(f"Error at offset {offset}: {str(e)}")
       raise

Approximately 9981955 rows in comp_snapshot.wrds_csq_pit.
Total rows to process: 9981955
Processed 500000 rows out of 9981955 (5.01%)
Processed 1000000 rows out of 9981955 (10.02%)
Processed 1500000 rows out of 9981955 (15.03%)
Processed 2000000 rows out of 9981955 (20.04%)
Processed 2500000 rows out of 9981955 (25.05%)
Processed 3000000 rows out of 9981955 (30.05%)
Processed 3500000 rows out of 9981955 (35.06%)
Processed 4000000 rows out of 9981955 (40.07%)
Processed 4500000 rows out of 9981955 (45.08%)
Processed 5000000 rows out of 9981955 (50.09%)
Processed 5500000 rows out of 9981955 (55.10%)
Processed 6000000 rows out of 9981955 (60.11%)
Processed 6500000 rows out of 9981955 (65.12%)
Processed 7000000 rows out of 9981955 (70.13%)
Processed 7500000 rows out of 9981955 (75.14%)
Processed 8000000 rows out of 9981955 (80.14%)
Processed 8500000 rows out of 9981955 (85.15%)
Processed 9000000 rows out of 9981955 (90.16%)
Processed 9500000 rows out of 9981955 (95.17%)
Processed 9981955 ro

In [24]:
# Count rows
total_rows = duck_conn.execute("SELECT COUNT(*) AS total_rows FROM wrds_csq_pit").fetchdf()

# Count columns
total_columns = duck_conn.execute("""
    SELECT COUNT(*) AS total_columns
    FROM information_schema.columns
    WHERE table_name = 'wrds_csq_pit'
""").fetchdf()

# Combine results
print(f"Total Rows: {total_rows['total_rows'][0]}")
print(f"Total Columns: {total_columns['total_columns'][0]}")

Total Rows: 9981955
Total Columns: 672


In [25]:
duck_conn.close()
db.close()